In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, udf, year, month, dayofmonth, dayofweek, datediff, to_date, regexp_replace, length, unix_timestamp, from_unixtime, log
from pyspark.sql.types import DoubleType

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import re
import matplotlib.pyplot as plt
import pandas as pd
from constants import TEST_TRANSFORMED_DATA, TRAIN_TRANSFORMED_DATA

In [24]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("CO2 Emission ML Pipeline - Modelling") \
    .getOrCreate()

#reading df
TEST_TRANSFORMED_DF = spark.read.parquet(TEST_TRANSFORMED_DATA)
TRAIN_TRANSFORMED_DF = spark.read.parquet(TRAIN_TRANSFORMED_DATA)


In [25]:
# List of categorical columns 
categorical_columns = ["transport_mode", "project_location", "material_category", "supplier_location"]
# list of scaled columns
columns_to_scale = ["Quantity_Squared", "Distance_Covered_Squared", "Quantity_Distance_Interaction", "supplier_rating", "Transaction_Year", "Transaction_Month", "project_duration"]

#### Linear Regression


In [26]:
# initialize the Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="log_CO2_emission")

# train model
lr_model = lr.fit(TRAIN_TRANSFORMED_DF)

# predict on the test data
lr_predictions = lr_model.transform(TEST_TRANSFORMED_DF)

# evaluate the model
lr_evaluator = RegressionEvaluator(labelCol="log_CO2_emission", predictionCol="prediction", metricName="rmse")
lr_rmse = lr_evaluator.evaluate(lr_predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {lr_rmse}")

lr_evaluator = RegressionEvaluator(labelCol="log_CO2_emission", predictionCol="prediction", metricName="r2")
lr_r2 = lr_evaluator.evaluate(lr_predictions)
print(f"R-squared on test data = {lr_r2}")


23/11/14 00:23:00 WARN Instrumentation: [7dedc208] regParam is zero, which might cause numerical instability and overfitting.


Root Mean Squared Error (RMSE) on test data = 1.0529085351593626
R-squared on test data = 0.3258366813389463


#### Gradient Boosted Trees

In [27]:
# initialize GBTRegressor
gbt = GBTRegressor(featuresCol="features", labelCol="log_CO2_emission")

# create parameter grid 
gbt_paramGrid = (ParamGridBuilder()
                 .addGrid(gbt.maxDepth, [5, 10])
                 .addGrid(gbt.maxBins, [16])
                 .addGrid(gbt.maxIter, [10])
                 .build())

# initialize evaluator with the appropriate metric - rmse
gbt_evaluator = RegressionEvaluator(
    labelCol="log_CO2_emission", predictionCol="prediction", metricName="rmse"
)

# 5-fold CrossValidator
gbt_cv = CrossValidator(estimator=gbt,
                        estimatorParamMaps=gbt_paramGrid,
                        evaluator=gbt_evaluator,
                        numFolds=3)

# run cross-validation
gbt_cv_model = gbt_cv.fit(TRAIN_TRANSFORMED_DF)

# predict on the test data
gbt_cv_predictions = gbt_cv_model.transform(TEST_TRANSFORMED_DF)

# evaluate the model
gbt_cv_rmse = gbt_evaluator.evaluate(gbt_cv_predictions)
print(f"Root Mean Squared Error (RMSE) on test data with CV = {gbt_cv_rmse}")

# To evaluate R-squared
gbt_evaluator.setMetricName("r2")
gbt_cv_r2 = gbt_evaluator.evaluate(gbt_cv_predictions)
print(f"R-squared on test data with CV = {gbt_cv_r2}")

23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1013.0 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1064.1 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1063.7 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1064.1 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1064.8 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1065.9 KiB
23/11/14 00:23:14 WARN DAGScheduler: Broadcasting large task binary with size 1068.1 KiB
23/11/14 00:23:15 WARN DAGScheduler: Broadcasting large task binary with size 1072.8 KiB
23/11/14 00:23:15 WARN DAGScheduler: Broadcasting large task binary with size 1081.3 KiB
23/11/14 00:23:15 WARN DAGScheduler: Broadcasting large task binary with size 1097.9 KiB
23/11/14 00:23:15 WARN DAGScheduler: Broadcasting large task binary with size 1126.5 KiB
23/11/14 00:23:15 WAR

Root Mean Squared Error (RMSE) on test data with CV = 0.5764742370042398
R-squared on test data with CV = 0.7979105542889521


#### Random Forest Regressor

In [28]:
# initialize RandomForest regressor
rf = RandomForestRegressor(featuresCol="features", labelCol="log_CO2_emission")

# parameter grid 
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 30])  # List of trees to test
             .addGrid(rf.maxDepth, [5, 10])    # List of maximum depths to test
             .addGrid(rf.maxBins, [32])        # List of bins to test
             .build())

# evaluator for the cross-validation
evaluator = RegressionEvaluator(labelCol="log_CO2_emission", predictionCol="prediction", metricName="rmse")

# crossValidator requires the same evaluator used to evaluate the model
cv = CrossValidator(estimator=rf,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)  # Number of folds for cross-validation

# run cross-validation
cv_model = cv.fit(TRAIN_TRANSFORMED_DF)

# Use the best model found to make predictions on the test data
cv_predictions = cv_model.transform(TEST_TRANSFORMED_DF)

# evaluate best model
cv_rmse = evaluator.evaluate(cv_predictions)
print(f"Root Mean Squared Error (RMSE) on CV test data = {cv_rmse}")

#R-squared eevaluation
evaluator.setMetricName("r2")
cv_r2 = evaluator.evaluate(cv_predictions)
print(f"R-squared on CV test data = {cv_r2}")

# Get best model
best_rf_model = cv_model.bestModel

23/11/14 00:23:48 WARN DAGScheduler: Broadcasting large task binary with size 1393.3 KiB
23/11/14 00:23:51 WARN DAGScheduler: Broadcasting large task binary with size 1303.6 KiB
23/11/14 00:23:52 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
23/11/14 00:23:53 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
23/11/14 00:23:58 WARN DAGScheduler: Broadcasting large task binary with size 1381.6 KiB
23/11/14 00:24:01 WARN DAGScheduler: Broadcasting large task binary with size 1306.0 KiB
23/11/14 00:24:02 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
23/11/14 00:24:03 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
23/11/14 00:24:07 WARN DAGScheduler: Broadcasting large task binary with size 1399.9 KiB
23/11/14 00:24:10 WARN DAGScheduler: Broadcasting large task binary with size 1307.2 KiB
23/11/14 00:24:11 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
23/11/14 00:24:12 WARN DAGScheduler:

Root Mean Squared Error (RMSE) on CV test data = 0.5538786257349492
R-squared on CV test data = 0.8134423603666683


#### Training Outcome
The results show that both the Gradient Boosted Trees (GBT) and Random Forest Regressor models have performed significantly better than the Linear Regression model in terms of both RMSE and R-squared. The RMSE is lower for the GBT and Random Forest models, indicating better accuracy, and the R-squared values are significantly higher, suggesting that these models explain a much greater proportion of the variance in the data.

#### Feature importance Analysis
Analyzing feature importance is a crucial step in understanding and interpreting the model.

##### Random Forest Feature Importance
I'm taking Random Forest Regressor as my model of choice
With PySpark we can use the attribute 'featureImportances' 

In [29]:
importances = best_rf_model.featureImportances.toArray()

# Start with an empty list for feature names
feature_names = []

# Add names for the one-hot encoded categorical features
# You need to know the number of categories in each categorical feature after one-hot encoding
for categoricalCol in categorical_columns:
    # Assuming we know the number of categories for each column (replace with actual number)
    num_categories = TRAIN_TRANSFORMED_DF.select(categoricalCol + "Vec").head()[0].size
    feature_names += [f"{categoricalCol}_{i}" for i in range(num_categories)]

# Add names for the scaled numerical features
feature_names += columns_to_scale

# Add the log-transformed features if they are also included
feature_names.append("log_project_budget")

# The length of feature names should now match the length of importances
assert len(feature_names) == len(importances), f"Length of feature names ({len(feature_names)}) does not match the number of importances ({len(importances)})"

# Now you can match the importances to the feature names
named_importances = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

# Print the feature importances
for name, importance in named_importances:
    print(f"{name}: {importance}")


Quantity_Distance_Interaction: 0.6167063311225728
Quantity_Squared: 0.19594874684452945
Distance_Covered_Squared: 0.12366560907271244
log_project_budget: 0.014118016637641388
Transaction_Month: 0.011746204243516654
supplier_rating: 0.00670079278111016
project_duration: 0.005178558064682412
Transaction_Year: 0.004365846588385062
project_location_1: 0.002889254376436291
transport_mode_0: 0.002867409315901708
transport_mode_1: 0.0024417834859516373
material_category_1: 0.002415101831442552
supplier_location_0: 0.0023232044778321043
project_location_2: 0.0022809302618376375
material_category_0: 0.002238708897005056
supplier_location_1: 0.0021383195204839793
project_location_0: 0.0019751824779586213
